# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [76]:
import os
import glob
import json
import psycopg2
import pandas as pd
from sql_queries import *

In [47]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=admin")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [48]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [49]:
song_files = get_files("data/song_data")

In [50]:
filepath = song_files[0]

In [51]:
df = pd.DataFrame([pd.read_json(filepath, typ='series', convert_dates=False)])
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARGCY1Y1187B9A4FA5,36.16778,-86.77836,"Nashville, TN.",Gloriana,SOQOTLQ12AB01868D0,Clementina Santafè,153.33832,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [52]:
song_data = (df.values[0])
song_data = (song_data[6], song_data[7], song_data[1], song_data[9], song_data[8])
song_data

('SOQOTLQ12AB01868D0',
 'Clementina Santafè',
 'ARGCY1Y1187B9A4FA5',
 0,
 153.33832)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [55]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [53]:
artist_data = (df.values[0])
artist_data = (artist_data[1], artist_data[5], artist_data[4], artist_data[2], artist_data[3])
artist_data

('ARGCY1Y1187B9A4FA5', 'Gloriana', 'Nashville, TN.', 36.16778, -86.77836)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [54]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [56]:
log_files = get_files('data/log_data')

In [62]:
filepath = log_files[0]

In [64]:
filepath

'/home/nader/Projects/Udacity-Data-Engineering-Projects/Data-Modeling-with-Postgres/data/log_data/2018/11/2018-11-25-events.json'

In [65]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,matchbox twenty,Logged In,Jayden,F,0,Duffy,177.65832,free,"Seattle-Tacoma-Bellevue, WA",PUT,NextSong,1.540146e+12,846,Argue (LP Version),200,1543109954796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
1,The Lonely Island / T-Pain,Logged In,Jayden,F,1,Duffy,156.23791,free,"Seattle-Tacoma-Bellevue, WA",PUT,NextSong,1.540146e+12,846,I'm On A Boat,200,1543110131796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
2,None,Logged In,Jayden,F,2,Duffy,NaN,free,"Seattle-Tacoma-Bellevue, WA",GET,Home,1.540146e+12,846,None,200,1543110132796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
3,None,Logged In,Jayden,F,3,Duffy,NaN,free,"Seattle-Tacoma-Bellevue, WA",GET,Settings,1.540146e+12,846,None,200,1543110168796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
4,None,Logged In,Jayden,F,4,Duffy,NaN,free,"Seattle-Tacoma-Bellevue, WA",PUT,Save Settings,1.540146e+12,846,None,307,1543110169796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [66]:
df = df[df['page'] == 'NextSong'].astype({'ts': 'datetime64[ms]'})
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,matchbox twenty,Logged In,Jayden,F,0,Duffy,177.65832,free,"Seattle-Tacoma-Bellevue, WA",PUT,NextSong,1.540146e+12,846,Argue (LP Version),200,2018-11-25 01:39:14.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
1,The Lonely Island / T-Pain,Logged In,Jayden,F,1,Duffy,156.23791,free,"Seattle-Tacoma-Bellevue, WA",PUT,NextSong,1.540146e+12,846,I'm On A Boat,200,2018-11-25 01:42:11.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
5,John Mayer,Logged In,Wyatt,M,0,Scott,275.27791,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,856,All We Ever Do Is Say Goodbye,200,2018-11-25 02:35:47.796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
7,10_000 Maniacs,Logged In,Wyatt,M,2,Scott,251.87220,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,856,Gun Shy (LP Version),200,2018-11-25 02:40:22.796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
8,Leona Lewis,Logged In,Chloe,F,0,Cuevas,203.88526,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,916,Forgive Me,200,2018-11-25 05:05:48.796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49


In [70]:
t = pd.Series(df['ts'], index=df.index)
t.head()

0   2018-11-25 01:39:14.796
1   2018-11-25 01:42:11.796
5   2018-11-25 02:35:47.796
7   2018-11-25 02:40:22.796
8   2018-11-25 05:05:48.796
Name: ts, dtype: datetime64[ns]

In [71]:
time_data = []
for data in t:
    time_data.append([data, data.hour, data.day, data.weekofyear, data.month, data.year, data.day_name()])

column_labels = ["timestamp", "hour", "day", "weelofyear", "month", "year", "weekday"]

In [72]:
time_df = pd.DataFrame.from_records(data=time_data, columns=column_labels)
time_df.head()

,timestamp,hour,day,weelofyear,month,year,weekday
0,2018-11-25 01:39:14.796,1,25,47,11,2018,Sunday
1,2018-11-25 01:42:11.796,1,25,47,11,2018,Sunday
2,2018-11-25 02:35:47.796,2,25,47,11,2018,Sunday
3,2018-11-25 02:40:22.796,2,25,47,11,2018,Sunday
4,2018-11-25 05:05:48.796,5,25,47,11,2018,Sunday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [73]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [81]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [82]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [84]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [85]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.